In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# imbalanced learning
import imblearn
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
import json
import numpy as np

In [ ]:
text = []
label = []
N_FEATURES = 20000
# 0 machine model1, 1 human
with open("drive/MyDrive/data/domain1_train.json") as f:
    for line in f:
        # read line by line
        data = json.loads(line)

        # add values
        text.append(data["text"])
        label.append(data["label"])

In [ ]:
# 2-8 machine 9 human
with open("drive/MyDrive/data/domain2_train.json") as f:
    for line in f:
        # read line by line
        data = json.loads(line)

        # add values
        text.append(data["text"])
        if "model" in data.keys():
            label.append(data["model"]+2)
        else:
            label.append(9)

In [ ]:
# vector_sample = np.arange(5000)
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def toStr(n):
   return str(n)

# Create a Vectorizer Object
vectorizer = TfidfVectorizer(preprocessor= toStr, analyzer="word", token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 3), max_features=N_FEATURES)

vectorizer.fit(text)

# Printing the identified Unique words along with their indices
# print("Vocabulary: ", vectorizer.vocabulary_)

# Encode the Document
vector = vectorizer.transform(text)
# Summarizing the Encoded Texts
# print("Encoded Document is:")
# print(vector.toarray())

In [ ]:
X = vector.toarray()
y = np.array(label).ravel()

print("Shape of X:", X.shape)
print("Shape of y", y.shape)

Shape of X: (34400, 20000)
Shape of y (34400,)


In [ ]:
# Split data to train and test
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X, y)

for i in range(len(y_train_ros)):
    if y_train_ros[i] == 1 or y_train_ros[i] == 9:
        y_train_ros[i] = 1
    else:
        y_train_ros[i] = 0

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_ros, y_train_ros)


In [ ]:
for i in range(len(y_test)):
    if y_test[i] == 1 or y_test[i] == 9:
        y_test[i] = 1
    else:
        y_test[i] = 0

In [ ]:
from collections import Counter

Counter(y_train_rus)

Counter({0: 15590, 1: 15590})

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipe = make_pipeline(StandardScaler(), LogisticRegression(random_state=0))
model = pipe.fit(X_train_rus, y_train_rus)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
import os
import pandas as pd

# Cross-validation scores
cv_score = cross_val_score(model, X_test, y_test, cv=5)
print(cv_score)

# Confusion Matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
tn, fp, fn, tp = cm.ravel()

# Record on csv
df = pd.DataFrame({
    'n_features': N_FEATURES,
     '5-fold_cv_score': [sum(cv_score)/len(cv_score)],
     'tn': [tn],
     'fp': [fp],
     'fn': [fn],
     'tp': [tp]
     })
path = "./LR_Records.csv"
exists= not os.path.exists(path)
df.to_csv(path, index=False, header=exists, mode='a')

[0.81104651 0.8190407  0.79869186 0.82194767 0.81177326]
[[3605  884]
 [ 486 1905]]


In [ ]:
text = []

with open("drive/MyDrive/data/test_set.json") as f:
    for line in f:
        # read line by line
        data = json.loads(line)

        # add values
        text.append(data["text"])

In [ ]:
# Encode the Document
vector = vectorizer.transform(text)

# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

X = vector.toarray()

print("Shape of X:", X.shape)

Encoded Document is:
[[0.31267836 0.07291718 0.         ... 0.         0.         0.        ]
 [0.01135859 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.19333699 0.         0.         ... 0.         0.         0.        ]
 [0.17332521 0.         0.         ... 0.         0.         0.        ]]
Shape of X: (1000, 25000)


In [ ]:
import pandas as pd
y_result = model.predict(X)

print(y_result)


[0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 1 0 1
 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0
 1 0 1 1 1 0 0 1 0 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1
 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0
 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0
 0 0 1 1 1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 1 0 0 1 0 0 1 1 0 1 0 1
 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1 1
 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0
 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1
 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 0
 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0
 1 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0
 0 1 1 0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1
 0 0 0 0 1 0 0 1 1 0 0 1 

In [ ]:
y_1 = list(y_result)
y_1.count(1)

424

In [ ]:

result = pd.DataFrame({"class":y_result}).reset_index().rename(columns = {'index':'id'})
result.to_csv("./result_LR_predictions-I.csv", index=False)